## Incident Forecast for IT Infrastructure

This is a trainable algorithm which trains on user provided historical incident data and builds & saves a deep learning model which is a representation of the historical data,once the model is generated, the solution can be used to predict next three most probable incidents. 

Mphasis DeepInsights Incident forecasting is a solution to forecast most probable next three incidents/errors in IT infrastructure. The solution helps in better incident management and lower downtime through proactive actions.


This sample notebook shows you how to deploy Mphasis InfraGraf Incident Forecast for IT Infrastructure using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Incident Forecast for IT Infrastructure. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
    3. Make sure to install below mentioned python libraries before running this notebook
    
#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Usage Instruction](#2.-Usage-Instruction)
3. [Training](#3.-Training) 
4. [Sample Data](#4.-Sample-Data)
5. [Perform batch inference](#5.-Perform-batch-inference)
6. [Create an endpoint and perform real-time inference](#6.-Create-an-endpoint-and-perform-real-time-inference)
7. [Clean-up](#7.-Clean-up)
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the algorithm listing page **Incident Forecast for IT Infrastructure**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### 2. Usage Instruction

The deployed solution has these 2 steps:
1: The system trains on user provided historical incident data and builds & saves a deep learning model which is a representation of the historical data.
2: Once the model is generated, the solution can be used to predict next three most probable incidents. 
To achieve this end, the solution deploys the following 2 APIs over AWS Sagemaker:

1.Training API: The solution requires historical consecutive incidents along with contextual information as input for training the model.
2.Testing API: The solution requires recent historical incidents which would be used for forecasting next three incidents along with contextual information as input (same as training API).
## Input
Supported content types: `text/csv`
** Following are the mandatory inputs for both the APIs:**
1.	Incident Number: Unique ID for each incident
2.	Incident Created Date: Date (MM/DD/YYYY) when Incident occurred.
3.	Configuration: Category of Incident that occurred.

Note: 
1.	Two separate csv input files are required for training and testing.
2.	For better results please provide at least 2000 consecutive historical incidents for training API.
3.	Please provide at least 15 recent historical consecutive incidents for test API to forecast next three incidents.
  
## Output
• Content type: ` text/csv`
** The solution generates the following outputs:**
Output contains three most probable forecasted incidents.

## Invoking endpoint

### AWS CLI Command
If you are using real time inferencing, please create the endpoint first and then use the  following command to invoke it:

``` bash 
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/csv' --region us-east-2 output.csv
```
Substitute the following parameters:
* `"model-name"` - name of the inference endpoint where the model is deployed
* `file_name` - input csv file name 
* `text/csv` - content type of the given input 
* `output.csv` - filename where the inference results are written to.

In [3]:
import base64 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image
import numpy as np
import pandas as pd
import cv2
from numpy import asarray

In [4]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [5]:
# S3 prefixes
common_prefix = "incident_forecasting"
training_input_prefix = common_prefix + "/training-input-data"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"

In [6]:
sagemaker_session = sage.Session()

In [7]:
TRAINING_WORKDIR = "data/training"

TRAINING_DATA = TRAINING_WORKDIR + "/Train_Data.csv"

In [8]:
TRAINING_WORKDIR = "data/training"

training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix)
print ("Training Data Location " + training_input)

Training Data Location s3://sagemaker-us-east-2-786796469737/incident_forecasting/training-input-data


### 3. Training 

In [9]:
import json
import time
from sagemaker.algorithm import AlgorithmEstimator

##### Algorithm ARN

In [10]:
algorithm_arn ='arn:aws:sagemaker:us-east-2:786796469737:algorithm/incident-prediction'

In [11]:
algo = AlgorithmEstimator(
    algorithm_arn=algorithm_arn,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    base_job_name='incident-prediction')

In [12]:
print ("Now run the training job using algorithm arn %s in region %s" % (algorithm_arn, sagemaker_session.boto_region_name))
algo.fit({'training': training_input})

Now run the training job using algorithm arn arn:aws:sagemaker:us-east-2:786796469737:algorithm/incident-prediction in region us-east-2
2021-01-27 11:19:45 Starting - Starting the training job...
2021-01-27 11:20:07 Starting - Launching requested ML instancesProfilerReport-1611746384: InProgress
......
2021-01-27 11:21:08 Starting - Preparing the instances for training...
2021-01-27 11:21:42 Downloading - Downloading input data
2021-01-27 11:21:42 Training - Downloading the training image......
2021-01-27 11:22:34 Training - Training image download completed. Training in progress.Starting the training.
Training complete.

2021-01-27 11:23:30 Uploading - Uploading generated training model
2021-01-27 11:24:10 Completed - Training job completed
ProfilerReport-1611746384: NoIssuesFound
Training seconds: 149
Billable seconds: 149


### 4. Sample Data

In [13]:
import os
TRANSFORM_WORKDIR = "data/transform"
sample_data = pd.read_csv(TRANSFORM_WORKDIR + "/test.csv",index_col=0)
sample_data.head(10)

,Incident Created Date,Configuration
Incident Number,,
IM30296889,2020-09-09,Application Instance
IM30296604,2020-09-09,Incident Collector
IM30296892,2020-09-09,Application Instance
IM30295990,2020-09-09,Application
IM30297253,2020-09-09,Router
IM30296215,2020-09-09,Wireless Access Point
IM30294844,2020-09-09,Router
IM30294579,2020-09-09,Application Instance
IM30294247,2020-09-09,Application Instance


### 5. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [14]:
TRANSFORM_WORKDIR = "data/transform"
transform_input = sagemaker_session.upload_data(TRANSFORM_WORKDIR, key_prefix=batch_inference_input_prefix) + "/test.csv"
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/incident_forecasting/batch-inference-input-data/test.csv


In [ ]:
transformer = algo.transformer(1, 'ml.m5.large')
transformer.transform(transform_input, content_type='text/csv')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

#### Inspect the Batch Transform Output in S3

In [17]:
from urllib.parse import urlparse

parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = '{}/{}.out'.format(parsed_url.path[1:], 'test.csv')

s3_client = sagemaker_session.boto_session.client('s3')

response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)
response_bytes = response['Body'].read().decode('utf-8')

In [18]:
bucketFolder = transformer.output_path.rsplit('/')[3]

In [19]:
import boto3
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/' + 'test.csv' +'.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


### output

In [20]:
OUTPUT_WORKDIR = "data/output"

import pandas as pd

out_csv = pd.read_csv(OUTPUT_WORKDIR + "/output.csv",index_col=0)
out_csv.head(10)

,Configuration
0,Router
1,Router
2,Switch


### 6. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [21]:
model_name='incident-forecasting'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.xlarge'

##### Algorithm ARN

In [22]:
algorithm_arn ='arn:aws:sagemaker:us-east-2:786796469737:algorithm/incident-prediction'

#### A. Create an endpoint

In [23]:
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=algorithm_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = algo.deploy(1, 'ml.m5.xlarge',endpoint_name=model_name)

..........
-------------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [24]:
import pandas as pd
file_name = 'data/transform/test.csv'
data_frame = pd.read_csv(file_name)

#### C. Perform real-time inference

In [25]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'incident-forecasting' \
    --body fileb://$file_name \
    --content-type 'text/csv' \
    --region us-east-2 \
    output.csv

{
    "ContentType": "text/csv; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [26]:
out_csv = pd.read_csv("output.csv",index_col=0)
out_csv.head(10)

,Configuration
0,Router
1,Router
2,Switch


### 7. Clean-up

#### A. Delete the model

In [27]:
predictor.delete_endpoint()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

